In [2]:
%matplotlib inline

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from ipywidgets import interact, FloatSlider
import ipywidgets as widgets

# Deterministic SIR model
def SIR_demography(y, t, beta, gamma, Lambda, mu):
    S, I, R = y
    dS_dt = Lambda - beta * S * I / (S + I + R) - mu * S
    dI_dt = beta * S * I / (S + I + R) - gamma * I - mu * I
    dR_dt = gamma * I - mu * R
    return [dS_dt, dI_dt, dR_dt]

# Stochastic SIR model
def GA_with_demography(S0, I0, R0, beta, gamma, Lambda, mu, max_time):
    S, I, R = S0, I0, R0
    t = 0
    times = [t]
    S_values = [S]
    I_values = [I]
    R_values = [R]

    while t < max_time and I > 0:
        N = S + I + R

        # Calculate propensities
        a1 = beta * S * I / N        # Transmission
        a2 = gamma * I               # Recovery
        a3 = Lambda                  # Birth
        a4 = mu * S                  # Death of a susceptible
        a5 = mu * I                  # Death of an infected
        a6 = mu * R                  # Death of a recovered
        
        a0 = a1 + a2 + a3 + a4 + a5 + a6

        # Time until next event
        dt = -np.log(np.random.random()) / a0
        t += dt

        # Determine which event occurs
        r = np.random.random() * a0
        if r < a1:
            # Transmission event
            S -= 1
            I += 1
        elif r < a1 + a2:
            # Recovery event
            I -= 1
            R += 1
        elif r < a1 + a2 + a3:
            # Birth event
            S += 1
        elif r < a1 + a2 + a3 + a4:
            # Death of a susceptible
            S -= 1
        elif r < a1 + a2 + a3 + a4 + a5:
            # Death of an infected
            I -= 1
        else:
            # Death of a recovered
            R -= 1

        # Store results
        times.append(t)
        S_values.append(S)
        I_values.append(I)
        R_values.append(R)

    return times, S_values, I_values, R_values

# Define a function to measure the transient (deviation from deterministic equilibrium)
def measure_transient(stochastic, deterministic):
    return np.max(np.abs(stochastic - deterministic))

# Parameters
S0 = 950
I0 = 50
R0 = 0
beta = 0.3
gamma = 0.1
Lambda = 5  # birth rate
mu = 0.01  # death rate
max_time = 200
t = np.linspace(0, max_time, 1000)

# Main plotting function
def plot_transient(N, beta, gamma, Lambda, mu):
    # Adjust initial conditions based on N
    I0 = 10
    S0 = N - I0 - R0
    
    # Solve ODE for deterministic SIR
    solution = odeint(SIR_demography, [S0, I0, R0], t, args=(beta, gamma, Lambda, mu))
    S_det, I_det, R_det = solution.T
    
    # Plotting
    plt.figure(figsize=(10, 5))
    plt.plot(t, S_det, label="Susceptible (Deterministic)", color="orange", linestyle="--")
    plt.plot(t, I_det, label="Infectious (Deterministic)", color="red", linestyle="--")
    
    # Run and plot multiple stochastic simulations
    for _ in range(10):
        times, S_stoch, I_stoch, R_stoch = GA_with_demography(S0, I0, R0, beta, gamma, Lambda, mu, max_time)
        
        # Interpolate stochastic results onto common time grid
        S_stoch_interp = np.interp(t, times, S_stoch)
        I_stoch_interp = np.interp(t, times, I_stoch)
        
        plt.plot(t, S_stoch_interp, color="orange", alpha=0.2)
        plt.plot(t, I_stoch_interp, color="red", alpha=0.2)
    
    plt.title(f"N={N:.0f}, β={beta:.2f}, γ={gamma:.2f}, λ={Lambda:.3f}, μ={mu:.3f}")
    plt.xlabel("Time")
    plt.ylabel("Population")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Sliders for parameters
N_slider = FloatSlider(value=1000, min=100, max=5000, step=100, description='N')
beta_slider = FloatSlider(value=0.5, min=0.1, max=1, step=0.01, description='β')
gamma_slider = FloatSlider(value=0.05, min=0.01, max=0.1, step=0.01, description='γ')
Lambda_slider = FloatSlider(value=10, min=0.001, max=10, step=0.001, description='λ')
mu_slider = FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description='μ')

# Display the interactive plot
interact(plot_transient, N=N_slider, beta=beta_slider, gamma=gamma_slider, Lambda=Lambda_slider, mu=mu_slider);


interactive(children=(FloatSlider(value=1000.0, description='N', max=5000.0, min=100.0, step=100.0), FloatSlid…